In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import requests
import time
import random
import pyodbc
import numpy as np
from tqdm import tqdm
import dateutil.parser as parser
from selenium import webdriver
import schedule

In [2]:
## Delete old data
def run_delete(driver):
    url='https://app.powerbi.com/groups/40bfb9e8-b0f8-4dee-90e3-40aed6aed717/list/datasets'

    driver.get(url)
    time.sleep(10)

    username = 'powerbishare1@pharmacity.vn'
    password = 'Pmc@1234'

    signin = driver.find_element_by_css_selector('.column.medium-8.medium-centered .button')
    signin.click()
    time.sleep(5)

    usr_login = driver.find_element_by_id('i0116')
    usr_login.send_keys(username)
    time.sleep(3)

    next_btn = driver.find_element_by_id('idSIButton9')
    next_btn.click()
    time.sleep(3)

    psw_login = driver.find_element_by_id('i0118')
    psw_login.send_keys(password)
    time.sleep(3)

    login_btn = driver.find_element_by_id('idSIButton9')
    login_btn.click()
    time.sleep(3)

    ssi_btn = driver.find_element_by_id('idSIButton9')
    ssi_btn.click()
    time.sleep(10)

    search_bar = driver.find_element_by_id('mat-input-0')
    search_bar.send_keys('PMC')
    time.sleep(15)
    
    flag = True
    while flag:
        try:
            edit_btn = driver.find_element_by_css_selector('.pbi-glyph-edit')
            edit_btn.click()
            time.sleep(10)
            flag = False
        except:
            time.sleep(5)
    
    
    flag = True
    while flag:
        try:
            hda_btn_off = driver.find_element_by_css_selector('.inner-toggle-wrapper .ng-not-empty')
            coordinates = hda_btn_off.location_once_scrolled_into_view
            driver.execute_script(f"window.scrollTo({coordinates['x']}, {coordinates['y']});")
            time.sleep(3)
            hda_btn_off.click()
            time.sleep(3)
            flag = False
        except:
            time.sleep(5)

    confirm_btn = driver.find_element_by_class_name('group2-option3-background-active')
    confirm_btn.click()
    time.sleep(3)
    driver.get(driver.current_url)
    time.sleep(3)
    driver.refresh()
    time.sleep(15)
    
    flag = True
    while flag:
        try:
            edit_btn = driver.find_element_by_css_selector('.pbi-glyph-edit')
            edit_btn.click()
            time.sleep(20)
            flag = False
        except:
            time.sleep(5)

    
    flag = True
    while flag:
        try:
            hda_btn_on = driver.find_element_by_css_selector('.inner-toggle-wrapper .ng-empty')
            coordinates = hda_btn_on.location_once_scrolled_into_view
            driver.execute_script(f"window.scrollTo({coordinates['x']}, {coordinates['y']});")
            time.sleep(3)
            hda_btn_on.click()
            time.sleep(3)
            flag = False
        except:
            time.sleep(5)

    confirm_btn = driver.find_element_by_class_name('group2-option3-background-active')
    confirm_btn.click()
    time.sleep(5)


##class for data_generation
def run_query(query):
    result = []

    server = 'SGPMCAXDB03'
    database = 'PHARMACITY_BI_CALCULATE'
    username = 'pmcbiml'
    password = 'Pmc@1234'
    driver = '{ODBC Driver 17 for SQL Server}'
    cnxn = pyodbc.connect('DRIVER=' + driver +
                          ';SERVER=' + server +
                          ';DATABASE=' + database +
                          ';UID=' + username +
                          ';PWD=' + password
                          )
    
    
    df = pd.read_sql(query, cnxn)
    return df

In [3]:
# schedule.every(30).minutes.do(execute)

# while True:
#     schedule.run_pending()
#     time.sleep(1)

In [4]:
# import mysql.connector

# mydb = mysql.connector.connect(
# host="pharmacity-cluster.cluster-ro-cfme3xwcf2ka.ap-southeast-1.rds.amazonaws.com",
# database="pharmacity",
# user="pmcbi",
# password="Pharmacity@123!",
# port="3306"
# )
# mycursor = mydb.cursor()
# # Read the last reviews time from file:

# with open("maxtime.txt", "r") as infile:
#     maxlasttime = infile.read()

# # Check if there are new reviews
# query_str = f"SELECT * from VIEWBI_CUSTOMERRATES WHERE MODIFIEDDATETIME > '{maxlasttime}' ORDER BY MODIFIEDDATETIME DESC"
# mycursor.execute(query_str)

# reviews = []
# for x in mycursor:
#     reviews.append(x)

# if reviews:
#     msg_lst = []
#     time_lst = []
#     for x in reviews:
#         tmp = x[1] + ' ' + x[2] + ' ' + x[3] + ' ' + x[4] + ' ' + x[5] + ' ' + x[6]
#         msg_lst.append(tmp)d 
#         time_lst.append(x[1])

#     msg = '\n\n'.join(str(x) for x in msg_lst)
#     msg = "```\n" + msg + "```"
#     print(msg)

In [ ]:
REST_API_URL = 'https://api.powerbi.com/beta/2b45ec9b-3ea3-443d-a456-008e75294b76/datasets/3f387e3e-d083-4edd-b7df-e161876e9322/rows?key=jjG%2BcP7KeKEWzdgB%2BYny0f96UhOglL4Z7%2Fz6bKyY%2FK0ei35nIfcZnErRW1NRU0HngaHbDcIU18MbnNNMpBsUzw%3D%3D'

while True:
    try:
        query = f"SELECT RECID, DATEID, MODIFIEDDATETIME,HOUR,WAREHOUSEID,STAFFNAME,ABCTRANSACTION,TOPPRODUCT,LINECOUNT,PICKINGSTATUS FROM VIEW_STREAMINVENTPICKING"
        df = run_query(query)

        if not df.empty:
            print(f"FOUND {df.shape[0]} ROWS, PREPARE TO UPLOAD.", end="\r")
            #Make a copy of df
            data_raw = df.copy()

            data_raw = data_raw.drop(columns=['MODIFIEDDATETIME'])
            data_raw['DATEID'] = data_raw['DATEID'].apply(lambda x: x.isoformat())
            df_dict = {n: data_raw.iloc[n:n+10000, :] for n in range(0, len(data_raw), 10000)}

            driver = webdriver.Chrome(executable_path='chromedriver')
            run_delete(driver)
            driver.quit()
            time.sleep(2)

            #Push 10,000 rows per time
            for k, v in df_dict.items():
                data_json = bytes(v.to_json(orient='records'), encoding='utf-8')

                req = requests.post(REST_API_URL, data_json)
                print(f"Posted {v.shape[0]} rows to PowerBI at {datetime.now()}", end="\r")

    except Exception as e:
        print(e)
        driver.quit()
        
    finally:
        time.sleep(1800)